# COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University


In [ ]:
import pandas as pd
import pycountry
import datetime
from functools import reduce
import re
import numpy as np
from collections import ChainMap

In [ ]:
# papermill parameters
output_folder = "../output/"

In [ ]:
endpoint = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/{}.csv"
datasets = {
    "time_series_covid19_confirmed_US": "Confirmed",
    "time_series_covid19_deaths_US": "Deaths",
    "time_series_covid19_recovered_global": "Recovered",
    "time_series_covid19_confirmed_global": "Confirmed",
    "time_series_covid19_deaths_global": "Deaths"    
}

### Retrieve data

In [ ]:
res = {}
for data, case in datasets.items():
    df = pd.read_csv(endpoint.format(data)).rename(columns={
        "Long_": "Long",
        "Lat_": "Lat",
        "Country/Region": "Country_Region", 
        "Province/State": "Province_State"
    })
    if "_US" in data:
        df = df.set_index(["Country_Region", "Province_State", "Admin2"])
    else:
        df = df.set_index(["Country_Region", "Province_State"])

    df = df.add_suffix(f"_{case}")
    res[data] = df
        


### Merge global data

In [ ]:
c_glob = res["time_series_covid19_confirmed_global"]
d_glob = res["time_series_covid19_deaths_global"]
r_glob = res["time_series_covid19_recovered_global"]

In [ ]:
glob_merged = c_glob.merge(d_glob.drop(columns=["Lat_Deaths", "Long_Deaths"]), left_on=["Country_Region", "Province_State"], right_on=["Country_Region", "Province_State"], how="outer")
glob_merged = glob_merged.merge(r_glob.drop(columns=["Lat_Recovered", "Long_Recovered"]), left_on=["Country_Region", "Province_State"], right_on=["Country_Region", "Province_State"], how="outer")
glob_merged["Admin2"] = None
glob_merged = glob_merged.set_index(["Admin2"], append=True)
glob_merged = glob_merged.rename(columns={"Lat_Confirmed": "Lat", "Long_Confirmed": "Long"})

### Merge US data

In [ ]:
c_us = res["time_series_covid19_confirmed_US"]
d_us = res["time_series_covid19_deaths_US"]

In [ ]:
us_merged = c_us.merge(d_us.drop(columns=["UID_Deaths", "iso2_Deaths", "iso3_Deaths", "code3_Deaths","FIPS_Deaths", "Lat_Deaths", "Long_Deaths", "Combined_Key_Deaths", "Population_Deaths"]), left_on=["Country_Region", "Province_State", "Admin2"], right_on=["Country_Region", "Province_State", "Admin2"])
us_merged = us_merged.rename(columns={
    "UID_Confirmed": "UID", 
    "iso2_Confirmed": "iso2", 
    "iso3_Confirmed": "iso3",
    "code3_Confirmed": "code3",
     "FIPS_Confirmed": "FIPS",
     "Lat_Confirmed": "Lat",
     "Long_Confirmed": "Long",
     "Combined_Key_Confirmed": "Combined_Key"
})

### Concat dataframes

In [ ]:
df = pd.concat([glob_merged, us_merged], sort=True)
df = df.reset_index()

### Unpivot tables

In [ ]:
df = df.melt(id_vars=list(filter(lambda col: not re.match(r"\d{0,2}/\d{0,2}/2\d_", col), df.columns)), var_name="Date", value_name="Cases")


### Retrieve Case Type, Parse Date

In [ ]:
df["Case_Type"] = df["Date"].str.extract(r"\d{0,2}/\d{0,2}/2\d_(\w+)")
df["Date"] = df["Date"].str.extract(r"(\d{0,2}/\d{0,2}/2\d)_\w+")
df["Date"] = pd.to_datetime(df["Date"])

### Data cleansing

In [ ]:
df = df.drop(columns=["UID", "code3", "iso2", "iso3", "Combined_Key"])

### Resolve FIPS padding

In [ ]:
df["FIPS"] = df["FIPS"].astype(str)
df["FIPS"] = df["FIPS"].replace(r"nan", "", regex=True)
df["FIPS"] = df["FIPS"].replace(r"\.0", "", regex=True)
df["FIPS"] = df["FIPS"].replace(r"\.","", regex=True)
df['FIPS'].loc[df['FIPS'] != ''] = df['FIPS'].str.zfill(5)
df['FIPS'] = df['FIPS'].replace(r'^(0{3,})(\d{2})$', r'\g<2>\g<1>', regex=True)
df["FIPS"].loc[df["FIPS"] == ""] = np.nan

### Resolve County

In [ ]:
df = df.rename(columns={"Admin2": "County"})
df['County'] = df['County'].replace(r'(?i)unassigned', 'unassigned', regex=True) 

In [ ]:
changed_names = {
    "Holy See": "Holy See (Vatican City State)",
    "Vatican City": "Holy See (Vatican City State)",
    "Hong Kong SAR": "Hong Kong",
    "Iran (Islamic Republic of)": "Iran, Islamic Republic of",
    "Iran": "Iran, Islamic Republic of",
    "Macao SAR": "Macao",
    "Macau": "Macao",
    "Republic of Korea": "Korea, Republic of",
    "South Korea": "Korea, Republic of",
    "Korea, South": "Korea, Republic of",
    "Republic of Moldova": "Moldova, Republic of",
    "Russia": "Russian Federation",
    "Saint Martin": "Sint Maarten (Dutch part)",
    "St. Martin": "Sint Maarten (Dutch part)",
    "Taipei and environs": "Taiwan, Province of China",
    "Vietnam": "Viet Nam",
    "occupied Palestinian territory": "Palestine, State of",
    "West Bank and Gaza": "Palestine, State of",
    "Taiwan*": "Taiwan, Province of China",
    "Congo (Brazzaville)": "Congo",
    "Congo (Kinshasa)": "Congo, The Democratic Republic of the",
    "Gambia, The": "Gambia",
    "The Gambia": "Gambia",
    "Tanzania": "Tanzania, United Republic of",
    "US": "United States",
    "Curacao": "Curaçao",
    "Brunei": "Brunei Darussalam",
    "Cote d'Ivoire": "Côte d'Ivoire",
    "Moldova": "Moldova, Republic of",
    "The Bahamas": "Bahamas",
    "Venezuela": "Venezuela, Bolivarian Republic of",
    "Bolivia": "Bolivia, Plurinational State of",
    "East Timor": "Timor-Leste",
    "Cape Verde": "Cabo Verde",
    "US": "United States",
    "Laos": "Lao People's Democratic Republic",
    "Burma": "Myanmar"
}

def normalize_names(df):
    df["Country_Region"] = df["Country_Region"].replace(changed_names)
    df["Cases"] = df["Cases"].replace(np.nan,0).astype(int)
    return df   
    #return(df.groupby(by=["Country_Region","Province_State", "Date", "Case_Type"], as_index=False).agg({"Cases": "sum", "Long": "first", "Lat": "first"}))

In [ ]:
df = normalize_names(df)

In [ ]:
df.loc[df["Country_Region"] == "Diamond Princess", "Province_State"] = "Diamond Princess"
df.loc[df["Country_Region"] == "Diamond Princess", "Country_Region"] = "Cruise Ship"

### Resolve ISO3166_1 and ISO3166_2 codes

In [ ]:
df['ISO3166_1'] = None
df['ISO3166_2'] = None

In [ ]:
unique_countries = list(df["Country_Region"].unique())
country_map = dict(ChainMap(*list(map(lambda x: {x: pycountry.countries.get(name=x)}, unique_countries))))

In [ ]:
subdivisions = {}
for country, pyc in country_map.items():
    if pyc:
        df["ISO3166_1"].loc[df["Country_Region"] == country] = pyc.alpha_2
        subdivisions[pyc.alpha_2] = {k.name: k.code.replace(f"{pyc.alpha_2}-", "") for k in pycountry.subdivisions.get(country_code=pyc.alpha_2)} 

In [ ]:
for country_code, subdivs in subdivisions.items():
    if len(df.loc[(df["ISO3166_1"] == country_code) & (df["Province_State"].notna())]):
        for name, code in subdivs.items():
            df["ISO3166_2"].loc[(df["ISO3166_1"] == country_code) & (df["Province_State"] == name)] = code

### Add Last Update Date and Last Reported Flag

In [ ]:
df["Last_Update_Date"] = datetime.datetime.utcnow()
df['Last_Reported_Flag'] = df["Date"].max() == df["Date"]

In [ ]:
df.sample(20)

### Save to .csv

In [ ]:
df.to_csv(output_folder + "JHU_COVID_19_TIMESERIES.csv", index=False)